In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

# Census API Key
from config2 import api_key
c = Census(api_key, year=2018)

POSTGRES = {
    'user': 'postgres',
    'pw': 'YOUR PASS',
    'db': 'project_02',
    'host': 'localhost',
    'port': '5432',
}
engine = create_engine('postgresql://%(user)s:\
%(pw)s@%(host)s:%(port)s/%(db)s' % POSTGRES)


In [2]:
countries = pd.read_csv("CodesByCountry.csv")
countries.columns

Index(['code ', 'total', 'region ', 'sub region ', 'country '], dtype='object')

In [3]:
baseline = countries[['code ','country ']]
baseline = baseline.drop_duplicates(subset=['country '],keep='first')
baseline = baseline.dropna(subset=['country '])
countries_list = baseline['code '].values.tolist()
countries_list.append("B05006_001E")
countries_list_name = baseline['country '].values.tolist()
countries_list_name.append("TotalMigration")
country_list = []
for i in countries_list_name:
    a = str(i).replace(' ','')
    country_list.append(a)
    
countries_list_name = country_list
print(len(countries_list),len(countries_list_name))

137 137


In [4]:
population = ["B01003_001E"]

educational_degree_code = ["B06009_025E","B06009_026E","B06009_027E","B06009_028E","B06009_029E","B06009_030E"]
educational_degree_name = ["TotalEducation","School","HighSchool","College","Bachelor","Graduate"]

age_code = ["B06001_049E","B06001_050E","B06001_051E","B06001_052E","B06001_053E","B06001_055E",
            "B06001_056E","B06001_057E","B06001_058E","B06001_059E","B06001_060E"]
age_name = ["TotalAge","age_5","age_18","age_25","age_35","age_45","age_55","age_60","age_65","age_75","age_75more"]

income_code = ["B06010_045E","B06010_046E","B06010_047E","B06010_048E","B06010_049E","B06010_050E",
               "B06010_051E","B06010_052E","B06010_053E","B06010_054E","B06010_055E"]
income_name = ["TotalIncome","NoIncome","Income","inc_10000","inc_15000","inc_25000","inc_35000",
               "inc_50000","inc_65000","inc_75000","inc_75000more"]

sex_code = ["B06003_013E","B06003_014E","B06003_015E"]
sex_name = ["TotalSex","Male","Female"]


civil_status_code = ["B05007_001E","B05007_002E","B05007_003E","B05007_004E","B05007_005E","B05007_006E","B05007_007E",
                    "B05007_008E","B05007_009E","B05007_010E","B05007_011E","B05007_012E","B05007_013E"]
civil_stats_name = ["TotalCivil","2010","NotCitizen2010","2010Citizen","2000","NotCitizen2000","2000Citizen",
                   "1990","NotCitizen1990","1990Citizen","lower1990","lower1990NotCitizen","lower1990Citizen"]

In [5]:
data = population + educational_degree_code + age_code + income_code + sex_code + civil_status_code
data2 = educational_degree_name + age_name + income_name + sex_name + civil_stats_name
print(len(data),len(data2))

45 44


In [6]:
key=data[0]
for i in range(1,len(data)):
    key = key+","+data[i]
    
key2=countries_list[0]
for i in range(1,49):
    key2 = key2+","+countries_list[i]

key3=countries_list[49]
for i in range(50,98):
    key3 = key3+","+countries_list[i]

key4=countries_list[98]
for i in range(99,len(countries_list)):
    key4 = key4+","+countries_list[i]

In [7]:
census_data = c.acs5.get(("NAME", key), {'for': 'state:*'})
census_data = pd.DataFrame(census_data)

In [8]:
census_country1=c.acs5.get(("NAME", key2), {'for': 'state:*'})
census_country1=pd.DataFrame(census_country1)
census_country1 = census_country1.drop(columns=['NAME'])
census_country1.head()

,B05006_004E,B05006_005E,B05006_006E,B05006_007E,B05006_008E,B05006_009E,B05006_010E,B05006_011E,B05006_012E,B05006_014E,...,B05006_050E,B05006_051E,B05006_052E,B05006_053E,B05006_054E,B05006_055E,B05006_057E,B05006_058E,B05006_059E,state
0,5623.0,2533.0,2682.0,408.0,851.0,394.0,593.0,679.0,480.0,465.0,...,18562.0,1383.0,1818.0,2800.0,13462.0,26.0,332.0,886.0,30163.0,27
1,1740.0,576.0,942.0,222.0,185.0,91.0,10.0,63.0,27.0,64.0,...,2831.0,75.0,559.0,824.0,962.0,72.0,52.0,195.0,4436.0,28
2,5308.0,2583.0,2275.0,450.0,481.0,180.0,89.0,262.0,87.0,518.0,...,16861.0,820.0,2338.0,2017.0,7635.0,322.0,739.0,723.0,19703.0,29
3,1324.0,602.0,658.0,64.0,145.0,153.0,125.0,141.0,82.0,133.0,...,1313.0,98.0,114.0,535.0,827.0,0.0,0.0,45.0,170.0,30
4,1560.0,624.0,765.0,171.0,243.0,65.0,54.0,25.0,45.0,53.0,...,4196.0,209.0,296.0,974.0,2394.0,3.0,539.0,90.0,6496.0,31


In [9]:
census_country2=c.acs5.get(("NAME", key3), {'for': 'state:*'})
census_country2=pd.DataFrame(census_country2)
census_country2 = census_country2.drop(columns=['NAME'])
census_country2.head()

,B05006_060E,B05006_061E,B05006_062E,B05006_063E,B05006_064E,B05006_065E,B05006_066E,B05006_068E,B05006_069E,B05006_070E,...,B05006_108E,B05006_110E,B05006_111E,B05006_112E,B05006_113E,B05006_114E,B05006_115E,B05006_119E,B05006_120E,state
0,1766.0,672.0,2439.0,2457.0,763.0,238.0,1102.0,4716.0,667.0,24413.0,...,32.0,33.0,2575.0,12227.0,6436.0,1285.0,3696.0,1210.0,323.0,27
1,282.0,23.0,434.0,477.0,160.0,0.0,37.0,195.0,127.0,101.0,...,0.0,0.0,83.0,169.0,482.0,75.0,153.0,242.0,85.0,28
2,1589.0,316.0,1483.0,3737.0,364.0,320.0,647.0,840.0,496.0,894.0,...,89.0,0.0,1099.0,1205.0,2537.0,244.0,772.0,797.0,197.0,29
3,146.0,77.0,122.0,127.0,15.0,0.0,75.0,24.0,92.0,89.0,...,14.0,0.0,107.0,54.0,46.0,0.0,48.0,110.0,93.0,30
4,420.0,18.0,1737.0,298.0,96.0,140.0,965.0,9.0,165.0,510.0,...,0.0,0.0,279.0,114.0,730.0,0.0,1381.0,278.0,74.0,31


In [10]:
census_country3=c.acs5.get(("NAME", key4), {'for': 'state:*'})
census_country3=pd.DataFrame(census_country3)
census_country3 = census_country3.drop(columns=['NAME'])
census_country3.head()

,B05006_125E,B05006_126E,B05006_127E,B05006_128E,B05006_129E,B05006_130E,B05006_131E,B05006_132E,B05006_133E,B05006_134E,...,B05006_155E,B05006_156E,B05006_157E,B05006_158E,B05006_159E,B05006_161E,B05006_162E,B05006PR_013E,B05006_001E,state
0,6866.0,193.0,83.0,1248.0,99.0,1272.0,117.0,1121.0,1295.0,54.0,...,3711.0,1531.0,144.0,1344.0,671.0,11816.0,36.0,NaN,461758.0,27
1,3183.0,133.0,7.0,816.0,0.0,864.0,38.0,116.0,843.0,35.0,...,152.0,354.0,1.0,238.0,12.0,1656.0,6.0,NaN,69752.0,28
2,6785.0,95.0,31.0,2759.0,86.0,798.0,16.0,998.0,1412.0,29.0,...,221.0,1076.0,246.0,1125.0,278.0,5381.0,131.0,NaN,247788.0,29
3,414.0,0.0,0.0,112.0,0.0,142.0,0.0,40.0,69.0,18.0,...,24.0,251.0,0.0,103.0,60.0,3853.0,0.0,NaN,22685.0,30
4,4482.0,11.0,25.0,3413.0,0.0,268.0,9.0,240.0,265.0,12.0,...,21.0,693.0,29.0,629.0,28.0,1477.0,15.0,NaN,133424.0,31


In [11]:
data_df = census_data.merge(census_country1,left_on="state",right_on="state")
data_df = data_df.merge(census_country2,left_on="state",right_on="state")
data_df = data_df.merge(census_country3,left_on="state",right_on="state")
data_df.columns

Index(['NAME', 'B01003_001E', 'B06009_025E', 'B06009_026E', 'B06009_027E',
       'B06009_028E', 'B06009_029E', 'B06009_030E', 'B06001_049E',
       'B06001_050E',
       ...
       'B05006_154E', 'B05006_155E', 'B05006_156E', 'B05006_157E',
       'B05006_158E', 'B05006_159E', 'B05006_161E', 'B05006_162E',
       'B05006PR_013E', 'B05006_001E'],
      dtype='object', length=184)

In [12]:
names_list = ["State_Name"]+["Population"]+educational_degree_name + age_name + income_name + sex_name + civil_stats_name+["State"]
print(len(names_list))
for x in countries_list_name:
    names_list.append(x)
print(len(names_list))

47
184


In [13]:
print(len(names_list),len(data_df.columns))

184 184


In [14]:
dic={}
for i in range(0,len(data_df.columns)):
    dic[data_df.columns[i]]=names_list[i].lower()

data_df = data_df.rename(columns= dic)
data_df = data_df.set_index('state')
data_df = data_df.fillna(0)
data_df.head()

,state_name,population,totaleducation,school,highschool,college,bachelor,graduate,totalage,age_5,...,ecuador,guyana,peru,uruguay,venezuela,othersouthamerica,canada,othernorthernamerica,chinaexcludingtaiwan,totalmigration
state,,,,,,,,,,,,,,,,,,,,,
27,Minnesota,5527358.0,368926.0,94603.0,72829.0,78341.0,66278.0,56875.0,461758.0,5094.0,...,5919.0,3711.0,1531.0,144.0,1344.0,671.0,11816.0,36.0,0.0,461758.0
28,Mississippi,2988762.0,57881.0,18327.0,13983.0,10675.0,7863.0,7033.0,69752.0,555.0,...,217.0,152.0,354.0,1.0,238.0,12.0,1656.0,6.0,0.0,69752.0
29,Missouri,6090062.0,199304.0,42307.0,40849.0,37835.0,37119.0,41194.0,247788.0,2440.0,...,566.0,221.0,1076.0,246.0,1125.0,278.0,5381.0,131.0,0.0,247788.0
30,Montana,1041732.0,18523.0,2618.0,4288.0,4671.0,3626.0,3320.0,22685.0,77.0,...,39.0,24.0,251.0,0.0,103.0,60.0,3853.0,0.0,0.0,22685.0
31,Nebraska,1904760.0,106215.0,43309.0,22033.0,16682.0,12987.0,11204.0,133424.0,1578.0,...,145.0,21.0,693.0,29.0,629.0,28.0,1477.0,15.0,0.0,133424.0


In [15]:
data_df.to_sql(name='project', con=engine, if_exists='append', index=False)